In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import pyvo as vo

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [14]:

res = Simbad.query_object("hd283809")
# print(res.colnames)
# print(res['RA'].data, res['DEC'].data)

coordstr = '%s %s' % (res['RA'].data[0], res['DEC'].data[0])
print(coordstr)
coord = SkyCoord(coordstr, unit=(u.hourangle, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
print(coord)
print(coord.ra.degree, coord.dec.degree)

tap_service = vo.dal.TAPService("http://gaia.ari.uni-heidelberg.de/tap")
print(tap_service.tables)


04 41 24.7242 +25 54 48.391
<SkyCoord (ICRS): (ra, dec) in deg
    ( 70.3530175,  25.91344194)>
70.35301749999999 25.913441944444443


In [ ]:

tap_results = tap_service.search("SELECT TOP 10 * FROM  gaiadr1.gaia_source")

query_str = """SELECT * FROM gaiadr1.gaia_source
               WHERE CONTAINS(POINT('ICRS',ra,dec),
               CIRCLE('ICRS',%s,%s,0.01))=1""" % (coord.ra.degree,
                                                  coord.dec.degree)
print(query_str)
tap_results = tap_service.search(query_str)
#                                    CIRCLE('ICRS',70.3530175,25.91344194,0.01))=1

for tr in tap_results:
    print(tr['phot_g_mean_mag'])
    print(tr['parallax'], tr['parallax_error'])
    print(tr['phot_g_mean_flux'], tr['phot_g_mean_flux_error'])
    # print(tr.keys())
# gaiadr1.gaia_source

# SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',66.76957,26.10453,0.01))=1"


# r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
# r.pprint()

In [133]:
query_str = '''
WITH sample AS (
    SELECT TOP 1000 * FROM gdr2mock.main
    WHERE (distance(ra, dec, 265.48289546, -25.4975738)<2.5) AND (parallax > 5) AND (parallax < 20) )
SELECT ra, dec, parallax, parallax_error, a0, phot_g_mean_mag
FROM sample'''

#query_str = '''
#WITH sample AS (
#        SELECT TOP 10 * FROM gdr2mock.main
#        WHERE distance(ra, dec, 66.73, 75.87)<2)
#SELECT ra, dec, 1/parallax
#FROM sample'''



query_str = '''SELECT TOP 1000 * FROM gdr2mock.main'''

mockgaia_tap_service = vo.dal.tap.TAPService("http://dc.zah.uni-heidelberg.de/tap")
sync_tap_results = mockgaia_tap_service.run_async(query_str)

AttributeError: 'functools.partial' object has no attribute 'getCharacterStream'

In [132]:
sync_tap_results

<Table masked=True length=1000>
     source_id              ra         ...   nobs  index_parsec
                           deg         ...                     
       int64             float64       ... float32    int32    
------------------- ------------------ ... ------- ------------
4054557051362213952    264.23004090817 ...    23.0   1036206099
4054557051362213945 264.23146983905599 ...    23.0   1131192238
4054557051362213941 264.23311588806098 ...    23.0   1027190186
4054557051362213932 264.23358946929199 ...    23.0    927189199
4054557051362213930 264.23247716823198 ...    23.0   1125188000
4054557051362213907 264.23520570418498 ...    23.0    830184000
4054557051362213929 264.22865585903799 ...    23.0   1125188000
4054557051362213910   264.232479560765 ...    23.0    729185000
4054557051362213942 264.23310791088301 ...    23.0   1028191044
                ...                ... ...     ...          ...
4054553168711778308 264.35931094460398 ...    23.0    936179348
40545531

In [95]:
center = SkyCoord(2.5, 2.5, unit=u.deg, frame='galactic')

In [97]:
center.icrs

<SkyCoord (ICRS): (ra, dec) in deg
    ( 265.48289546, -25.4975738)>

In [134]:
vo.__version__

'0.7rc2'